<a href="https://colab.research.google.com/github/andiparjoleanu/FootballDetection/blob/main/DetectFootball.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Football Detection**

This Python script is designed to detect a football in video files, track its movement, and predict its trajectory. It automatically downloads a sample video (AVI format) for demonstration.

**1. Setup and libraries**  
Computer vision tasks are handled via the OpenCV (cv2) library, while NumPy is used for high-performance numerical operations. Once the environment is ready, this section downloads the required .avi file from a public Google Drive link using the system command gdown.

In [ ]:
import cv2
import numpy as np

file_id = '10vkVUVCUdw7lNy8YnmpPZ2fNpRqVFRXg'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'rgb.avi'

!gdown {url} -O {output}

Downloading...
From: https://drive.google.com/uc?id=10vkVUVCUdw7lNy8YnmpPZ2fNpRqVFRXg
To: /content/rgb.avi
100% 12.9M/12.9M [00:00<00:00, 21.0MB/s]


**3. The 2D detection**  
This section performs the 2D football detection. It begins by opening the input video stream via the provided path and extracting its properties (width, height, fps), which are necessary to properly format the final output video.

In [ ]:
#Open the video stream
video = cv2.VideoCapture(video_path)

if not video.isOpened():
    raise IOError("The video could not be opened.")

#Extract video properties
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video.get(cv2.CAP_PROP_FPS)

if 0 in (width, height, fps):
    cap.release()
    raise ValueError("The video file is empty or broken.")

#Initialize the VideoWriter, which builds the output video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

The program attempts to detect the football in every frame using the following structure:

**2. A. Preprocessing**  
The computer needs to eliminate noise from an image to accurately detect an object. We resize the image for faster computation and apply a filter to each pixel to remove high-frequency noise. This filtering process is called convolution, which involves multiplying the value of each pixel by the corresponding value in the filter's matrix. We choose a Gaussian Filter because the Gaussian Curve emphasizes the value of the central pixel, creating a spherical effect in the filtered area. Thus, the circular shape of the ball is preserved, while smoothing the edges of sharp, straight objects (like grass or light glares). We choose the size of the filter as 11x11, large enough to smooth tiny shapes like grass and small enough to avoid blurring a ball (usually larger than 20 pixels wide).

In [ ]:
def preprocess(frame):
    blurred_frame = cv2.GaussianBlur(frame, (11, 11), 0)
    return blurred_frame

**2. B. Color Segmentation**  
RGB images are hard to process in Computer Vision because of the mix between different colors and light. An HSV (Hue, Saturation, Value) map for any pixel is more convenient because it defines the default color of the pixel (the hue), the intensity of the color (the saturation), and the brightness (the value). The hue is a circular 2D palette. In color theory, the hue is a circle, which is 360 degrees. An 8-bit memory contains numbers up to 255, so the computer divides the circle in half, making the hue go from 0 to 179. Our ball is red, so we create an HSV mask which compares the pixel's color with the values of the red tones. According to the HSV scheme, the red tones sit between 0 and 10 (orange-reds) and also between 165 to 179 (pink-reds).

In [ ]:
def red_segmentation(blurred_frame):
    hsv = cv2.cvtColor(blurred_frame, cv2.COLOR_BGR2HSV)

    # Considering the most intense orange-reds tones in the range
    start_orange_red = np.array([0, 120, 70])
    end_orange_red = np.array([10, 255, 255])

    # The function creates a black screen where only orange-reds are white
    orange_red_mask = cv2.inRange(hsv, start_orange_red, end_orange_red)

    # Considering the most intense pink-red tones in the range
    start_pink_red = np.array([165, 120, 70])
    end_pink_red = np.array([179, 255, 255])

    # This function creates a black screen where only pink-reds are white
    pink_red_mask = cv2.inRange(hsv, start_pink_red, end_pink_red)

    # Keeps the pixels that are white in either mask
    full_mask = cv2.bitwise_or(orange_red_mask, pink_red_mask)

    return full_mask

In [ ]:
cv2.destroyAllWindows()

Width: 1280, Height: 720, FPS: 30.0
